In [ ]:
# 웹크롤링
from urllib import parse
from bs4 import BeautifulSoup
import requests

#### 도로 위험지수
![Alt text](image.png)

<br/>

👉️ <a href='https://taas.koroad.or.kr/web/shp/sbm/initRiskRoadFrcstSys.do?menuId=WEB_KMP_TAI_AFS'>출처</a>

In [ ]:
# 도로의 위험지수가 3이상인 값만 읽어들이는 함수

def danger_streat(rs_html):
    # xml값 읽어들이기
    rng = int(rs_html.find_all('totalCount')[0].text)
    ls = rs_html.find_all('line_string')
    av = rs_html.find_all('anals_value')
    ag = rs_html.find_all('anals_grd')
    
    # 태그 속 값만 추출 -> df
    col = ['x1', 'y1', 'x2', 'y2', 'danger', 'danger_rank', 'center_x', 'center_y']
    df_danger = pd.DataFrame([np.zeros(len(col))], columns=col)
    
    for i in range(rng):
        temp = ls[i].text[1:-1]
        temp = temp.replace(', ', ' ')
        temp = temp.split(' ')
        
        y1 = float(temp[0])
        x1 = float(temp[1])
        y2 = float(temp[2])
        x2 = float(temp[3])
        c_x = (x1+x2)/2
        c_y = (y1+y2)/2

        bb = float(av[i].text)
        cc = int(ag[i].text)
        df_dange.loc[i] = [x1, y1, x2, y2,bb,cc, c_x, c_y]
        
    # rank>=3, index재정렬
    df_danger = df_danger[df_danger['danger_rank']>=3]
    df_danger = df_danger.reset_index(drop=True)
    
    return df_danger

- 도로교통공사 홈페이지 이전문제로 고속도로목록 크롤링이 안되어 Dgdgr_Highway_CodeList.xlsx파일로 전달받은 후 사용

In [ ]:
### Open_API
url = 'http://taas.koroad.or.kr/data/rest/road/dgdgr/highway'
key = '인증키'

df_danger2 = None
a = pd.read_excel(file_path + 'Dgdgr_Highway_CodeList.xlsx')
a = a.fillna(method='ffill')

for name, ic_code in zip(a.iloc[:1000,0], a.iloc[:1000,1]):
    queryParams = f'?{parse.quote_plus("authKey")}={key}&' + parse.urlencode(
        {
            parse.quote_plus('frwyNm'): name,  #고속도로목록
            parse.quote_plus('frwySctnNm'): ic_code,  #고속도로구간목록
            parse.quote_plus('vhctyCd'): '01',  #차종조회
            parse.quote_plus('type'): 'xml'
        })

    request2 = requests.get(url + queryParams)

    rs_text = request2.text
    rs_html = BeautifulSoup(rs_text, "xml")
    
    time.sleep(2) #교통량이 많은 지역에서 오류발생 #sleep으로 for문 간격두고 실행

    # df만들기
    df_danger2 = pd.concat([df_danger2, danger_streat(rs_html)], ignore_index=True)

In [ ]:
# df -> csv 파일로 저장
df_danger2.to_csv('./data/danger.csv', index=False)